In [1]:
import numpy as np
import random

In [2]:
text = '01100100011101010100'
text

'01100100011101010100'

In [3]:
H = np.ones((1, 1), dtype=np.int8)
while len(H) < 8:
    tmp_H = H
    H = np.zeros([len(H) * 2] * 2, dtype=np.int8)
    H[:len(tmp_H), :len(tmp_H)] = H[len(tmp_H):, :len(tmp_H)] = H[:len(tmp_H), len(tmp_H):] = tmp_H
    H[len(tmp_H):, len(tmp_H):] = tmp_H * -1

print(H)

[[ 1  1  1  1  1  1  1  1]
 [ 1 -1  1 -1  1 -1  1 -1]
 [ 1  1 -1 -1  1  1 -1 -1]
 [ 1 -1 -1  1  1 -1 -1  1]
 [ 1  1  1  1 -1 -1 -1 -1]
 [ 1 -1  1 -1 -1  1 -1  1]
 [ 1  1 -1 -1 -1 -1  1  1]
 [ 1 -1 -1  1 -1  1  1 -1]]


In [4]:
H = np.array([[ 1,  1,  1,  1,  1,  1,  1,  1],
                [ 1, -1,  1, -1,  1, -1,  1, -1],
                [ 1,  1, -1, -1,  1,  1, -1, -1],
                [ 1, -1, -1,  1,  1, -1, -1,  1],
                [ 1,  1,  1,  1, -1, -1, -1, -1],
                [ 1, -1,  1, -1, -1,  1, -1,  1],
                [ 1,  1, -1, -1, -1, -1,  1,  1],
                [ 1, -1, -1,  1, -1,  1,  1, -1]], dtype=np.int8)

def encode(text):
    block_size = len(H) // 2
    result = ""
    if len(text) % block_size:
        text = "0" * (block_size - len(text) % block_size) + text

    for idx in range(len(text) // block_size):
        block = text[block_size * idx: block_size * (idx + 1)]

        row = H[int(block[1:], 2)]
        if int(block[0]):
            row -= row
        
        result += "".join(map(str, (row + 1) // 2))
    return result

def decode(text):
    block_size = len(H)
    result = ""
    if len(text) % block_size:
        text = "0" * (block_size - len(text) % block_size) + text
    

    for idx in range(len(text) // block_size):
        block = text[block_size * idx: block_size * (idx + 1)]
        v = np.array(list(map(lambda x: int(x) * 2 - 1, block)))
        F = np.dot(v, H)
        max_idx = np.argmax(np.abs(F))
        
        result += "1" if F[max_idx] < 0 else "0"

        result += "0" * (len(bin(block_size)) - len(bin(max_idx)) - 1) + bin(max_idx)[2:]

    return result

In [5]:
text, encode(text), decode(encode(text))

('01100100011101010100',
 '1100001111110000100101101010010111110000',
 '01100100011101010100')

In [7]:
encoded = encode(text)
for _ in range(10):
    error_pos = random.randint(0, len(encoded) - 1)

    decoded = decode(encoded[:error_pos] + str(["1", "0"].index(encoded[error_pos])) +encoded[error_pos + 1:])

    print(f"Error in {error_pos} bit\n\tDecoded: {decoded}\n\tIs right: {decoded == text}")

Error in 31 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 28 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 16 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 6 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 0 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 21 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 37 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 21 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 18 bit
	Decoded: 01100100011101010100
	Is right: True
Error in 22 bit
	Decoded: 01100100011101010100
	Is right: True
